In [1]:
#Crear requirements
#%pip install pipreqs
#%pipreqs 

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.3/798.3 kB 13.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.4/257.4 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.1/386.1 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.5 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13724 sha256=33c7984d6a91fe10ac3a2a360c210cb08b0932161ed4ebcb4df15af47e1dd20c
  Stored in directory: /home/karvala/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1

In [1]:
# Intalar pymupdf
# %pip install pymupdf

#importar fitz para usar pymupdf (fitz es pymupdf)
import fitz  

In [2]:
# Instalar e importar la biblioteca pendulum y datetime para trabajar con fechas
# %pip install pendulum
from datetime import datetime
import pendulum

# Definir timezone e idioma en la biblioteca pendulum
tz = pendulum.timezone('Europe/Paris')
pendulum.set_locale('es')

In [3]:
# Importar regex
import re

In [4]:
# Intalar e importar la biblioteca para trabajar con codigos postales
# %pip install pgeocode

import pgeocode

In [5]:
def formato_fecha(fecha):
    """ Convierte fechas en formato DD/MM/YYYY o "DD de MMMM YYYY al formato DD.MM.YYYY.
    Args:
        fecha (Date or String): fecha a cambiar de formato.
    Returns:
        dt_final (Date): devuelve la fecha con el formato cambiado.
    """  
    dt = ''
    try:
        datetime.strptime(fecha, "%d.%m.%Y")
        return fecha
    except ValueError:
       pass
        
    if '/' in fecha:
        dt = pendulum.from_format(fecha, 'DD/MM/YYYY')
    elif 'de' in fecha:
        dt = pendulum.from_format(fecha.replace(' de', ''), 'DD MMMM YYYY')
    else:
        dt_final = fecha
    try:
        dt_final = dt.format('DD.MM.YYYY')
    except Exception:
        dt_final = fecha

    return dt_final    


In [6]:
def separa_CP(direccion):
    """ Divide una string en una lista por un código de 5 dígitos (incluido en la lista devuelta).
    Args:
        direccion (String): línea de texto que incluya un código postal.
    Returns:
        lista (list): devuelve una lista separando por CP. 
    """
    return re.split(r"(\d{5})", direccion)

In [7]:
def extaer_pag_pdf(ruta, num):
    """ Extrae el texto de un pdf convirtiéndolo en una lista dónde cada línea del texto es un elenmento de la lista.
    Args:
        ruta (String): ruta al archivo pdf que se va a leer.
        num (int): número de la página del pdf que se quiere leer (empezando en 0).
    Returns:
        lista (list): devuelve una lista separando cada línea del texto como un elemento de la lista.
    """
    file = fitz.open(ruta)
    pymupdf_text = []

    for page in file:
        pymupdf_text.append(page.get_text())

    return pymupdf_text[num].split('\n')  



In [8]:
# Crear el diccionario que se convertirá en el archivo JSON final con los datos a conseguir
diccionario_final = {
    "nombre_cliente": "",
    "dni_cliente": "",
    "calle_cliente": "",
    "cp_cliente": "",
    "población_cliente": "",
    "provincia_cliente": "",
    "nombre_comercializadora": "",
    "cif_comercializadora": "",
    "dirección_comercializadora": "",
    "cp_comercializadora": "",
    "población_comercializadora": "",
    "provincia_comercializadora": "",
    "número_factura": "",
    "inicio_periodo": "",
    "fin_periodo": "",
    "importe_factura": "",
    "fecha_cargo": "",
    "consumo_periodo": "",
    "potencia_contratada": ""
}

In [9]:
def pdf_a_dicc_y_lista(lineas_pdf):
    """ De una lista de strings, se eliminan las líneas vacias y se crea un diccionario con los elementos de los que se pueden extraer pares key/valor mediante la separación por ':' y, por otro lado, se crea una lista con todos los datos para tenerlos ordenados.
    Args:
        lineas_pdf (list): lista de strigns.
    Returns:
        dict_lineas(dict): devuelve un diccionario con los pares key/valor de las líneas que incuian ':' y el resto de líneas con una key numerada.
        lista_lineas (list): devuelve una lista donde cada elemnto es una string (misma lista de args sin elementos vacíos).
    """
    dict_lineas = {}
    lista_lineas = []
    contador = 0
    for linea in lineas_pdf:
        linea = linea.strip()
        #Se eliminan lineas vacias (o de solo un espacio)
        if linea != '':
            lista_lineas.append(linea)
            #Se dividen las líneas por el primer ':' para separarlas en Key/valor en el diccionario
            if ':' in linea:
                lista_linea = linea.split(':', 1)
                dict_lineas[lista_linea[0]] = lista_linea[1].strip() 
            else:
                # En caso de no contar con ':' en la línea, se añade al diccionario con la key:'otro' más un número incremental
                contador += 1
                dict_lineas[f'otro{contador}'] = linea
    return dict_lineas, lista_lineas

In [10]:
def extraer_info(dict_lineas, lista_lineas):
    # Se obtienen todos los datos que se pueden extraer mediante las llaves del diccionario 
    for key in dict_lineas.keys():
        try:
            if 'nº' in key.lower() and 'factura' in key.lower():
                diccionario_final["número_factura"] = dict_lineas[key]
            if 'periodo' in key.lower() or 'período' in key.lower():
                lista_fechas = dict_lineas[key].replace('del', '').replace('de', '').split('(')[0]
                if 'al' in dict_lineas[key].lower():
                    fechas_consumo = lista_fechas.split(' al ')
                elif 'a' in dict_lineas[key].lower():    
                    fechas_consumo = lista_fechas.split(' a ')
                diccionario_final["inicio_periodo"] = formato_fecha(fechas_consumo[0].strip())
                diccionario_final["fin_periodo"] = formato_fecha(fechas_consumo[-1].strip())
            
            if 'potencia' in key.lower() and 'importe' not in key.lower():
                diccionario_final["potencia_contratada"] = dict_lineas[key].split()[0]      
            if 'titular' in key.lower() or 'nombre' in key.lower() :
                diccionario_final["nombre_cliente"] = dict_lineas[key]     
            if 'dirección' in key.lower():
                print(f"direccion: {key}")
                try:
                    lista_direccion = separa_CP(dict_lineas[key])
                    diccionario_final["calle_cliente"] = lista_direccion[0]
                    diccionario_final["cp_cliente"] = lista_direccion[1]
                    diccionario_final["población_cliente"] = lista_direccion[2]
                    if diccionario_final["calle_cliente"] != '' and diccionario_final["cp_cliente"] != '' and diccionario_final["población_cliente"] != '':
                        break
                    else:
                        continue 
                except Exception:
                    continue
        except Exception:
            continue
    # Se duplica el for en el caso del cif y el Nif para poder romper el loop después de la primera localización y asegurarnos que es el de la comercializadora y el cliente respectivamente
    for key in dict_lineas.keys():
        if 'fecha' in key.lower() and 'cargo' in key.lower():
            diccionario_final["fecha_cargo"] = formato_fecha(dict_lineas[key])
            break
    for key in dict_lineas.keys():
        if 'diccionario_final["fecha_cargo"]' == '' and 'fecha' in key.lower() and 'emisión' in key.lower():
                diccionario_final["fecha_cargo"] = formato_fecha(dict_lineas[key])
                break
            
    for key in dict_lineas.keys():
        try:
            if 'importe' in key.lower() and 'factura' in key.lower():
                print(dict_lineas[key])
                diccionario_final["importe_factura"] = dict_lineas[key].split()[0]
                break
        except Exception:
            continue    

    for key in dict_lineas.keys():
        if 'nif' in key.lower():
            diccionario_final["dni_cliente"] = dict_lineas[key]  
            break

    # Mediante un enumerate extraemos todos los datos que sabemos que estan cerca unos de otros en la lista.
    lista_consumo = []
    lista_direccion = []
    for indice, linea in enumerate(lista_lineas):

        if 'potencia' in linea.lower() and diccionario_final["potencia_contratada"] == '':
            if 'kw' in lista_lineas[indice+1].lower():
                diccionario_final["potencia_contratada"] = lista_lineas[indice+1].split()[0]  

        if diccionario_final["dni_cliente"] == '' or diccionario_final["dni_cliente"] == None or len(diccionario_final["dni_cliente"]) != 9:
            try:
                dni = re.search(r"[0-9]{8}[A-Za-z]", linea)
                diccionario_final["dni_cliente"] = dni.group()
            except Exception:
                continue
        
        if (diccionario_final["inicio_periodo"] == '' or diccionario_final["fin_periodo"] == '') and ('periodo' in linea.lower() or 'período' in linea.lower()):
            fechas = linea.replace('periodo', '').replace('período', '').replace('Periodo', '').replace('Período', '').replace('del', '').replace('de', '').strip()
            
            if 'a' in linea.lower():    
                fechas_consumo = fechas.split(' a ')
            elif 'al' in linea.lower():
                fechas_consumo = fechas.split(' al ')
            print(f"fechas consumo2: {fechas_consumo}")
    
            diccionario_final["inicio_periodo"] = formato_fecha(fechas_consumo[0].strip())
            diccionario_final["fin_periodo"] = formato_fecha(fechas_consumo[-1].strip())
            

            
            
        
        if 'dirección' in linea.lower() and 'suministro' in linea.lower() and diccionario_final["calle_cliente"] == '':
            try:
                lista_direccion.append(linea) 
                lista_direccion.append(lista_lineas[indice+1]) 
                direccion_suministro = ' '.join(lista_direccion)
                direccion_dividir = direccion_suministro.split(':', 1)
                direccion = direccion_dividir[1].strip().split(',')
                diccionario_final["calle_cliente"] = direccion[0].strip() 
                diccionario_final["población_cliente"] = direccion[1].strip()  
            except Exception:
                direccion_entera = linea.split(':', 1)[1]
                direccion = separa_CP(direccion_entera)
                diccionario_final["calle_cliente"] = direccion[0].strip() 
                diccionario_final["población_cliente"] = direccion[2].strip()  
        
        
    # Se duplican los bucles for para poder pararlos después de la primera aparición en aquella información repetida a lo largo del pdf y que puede dar problemas.
    for indice, linea in enumerate(lista_lineas):
        if ('importe' in linea.lower() and 'factura' in linea.lower()) or ('total' in linea.lower() and 'pagar' in linea.lower()) and (diccionario_final["importe_factura"] == '' or diccionario_final["importe_factura"] <= 0.0):
            try:
                importe = float(lista_lineas[indice-1].split()[0].replace(',','.'))
                diccionario_final["importe_factura"] = str(importe)
                
            except Exception:
                lista_importe = lista_lineas[indice].replace(',','.').split()
                print(f"lista_importe: {lista_importe}")
                for elemento in lista_importe:
                    try:
                        importe = float(elemento)
                        diccionario_final["importe_factura"] = str(importe).replace('.', ',')
                        break
                    except Exception:
                        continue
                
    for indice, linea in enumerate(lista_lineas):
        if 'cif' in linea.lower():
            diccionario_final["cif_comercializadora"] = linea.split()[-1].replace('.', '').strip()
            try:
                if len(diccionario_final["cif_comercializadora"]) != 9:
                    diccionario_final["cif_comercializadora"] = ''
                    diccionario_final["cif_comercializadora"] = lista_lineas[indice+1].replace('.', '').strip()
                break
            except Exception:
                continue
    for indice, linea in enumerate(lista_lineas):
        if 'cif' in linea.lower():
            try:                
                diccionario_final["nombre_comercializadora"] = lista_lineas[indice-1]
                nombre = re.search(r"^[\sáéíóúÁÉÍÓÚàÀÈÈÒòa-z,A-Z]+S\.[AL]\.", diccionario_final["nombre_comercializadora"])
                diccionario_final["nombre_comercializadora"] = nombre.group()
                for x in range(-2, 3):
                    direccion_comercializadora = separa_CP(lista_lineas[indice+x])
                    diccionario_final["cp_comercializadora"] = direccion_comercializadora[1].strip()
                    if len(direccion_comercializadora[1]) == 5: 
                        diccionario_final["dirección_comercializadora"] = direccion_comercializadora[0].strip()
                        break
                    else:
                        continue      
            except Exception:
                continue
    for indice, linea in enumerate(lista_lineas):
        if (diccionario_final["dirección_comercializadora"] == '' or len(diccionario_final["dirección_comercializadora"]) < 5) and 'social' in linea.lower():
            try:
                lista_comercializadora = linea.split(':', 1)
                nombre = re.search(r"^[\sa-z,A-Z]+S\.[AL]\.", lista_comercializadora[0])
                diccionario_final["nombre_comercializadora"] = nombre.group()

                lista_direccion_comercial = separa_CP(lista_comercializadora[-1])
                diccionario_final["dirección_comercializadora"] = lista_direccion_comercial[0].strip()
                diccionario_final["cp_comercializadora"] = lista_direccion_comercial[1]
                break
            except:
                continue

    for indice, linea in enumerate(lista_lineas):
        if 'cif' in linea.lower():        
            try:
                if diccionario_final["nombre_cliente"] == '':
                    diccionario_final["nombre_cliente"] = lista_lineas[indice+2]
                    if diccionario_final["calle_cliente"] == '':
                        diccionario_final["calle_cliente"] = lista_lineas[indice+3]
                    direccion_cliente = separa_CP(lista_lineas[indice+4])
                    break
            except Exception:
                continue        
                     
    for indice, linea in enumerate(lista_lineas):
        if 'consumo' in linea.lower() and 'p1' in linea.lower() and diccionario_final["consumo_periodo"] == '':
            try: 
                consumo = float(lista_lineas[indice+1].split('kWh')[0].strip())
                diccionario_final["consumo_periodo"] = consumo
                break 
            except Exception:
                continue

    for indice, linea in enumerate(lista_lineas):
        if 'consumo' in linea and (diccionario_final['consumo_periodo'] == '' or 'xx' in str(diccionario_final['consumo_periodo']).lower()):
            for x in range(1,5):
                if 'kwh' in lista_lineas[indice+x].lower():
                    try:
                        diccionario_final['consumo_periodo'] = int(linea.split()[0])
                        if diccionario_final['consumo_periodo'] != '' or 'xx' not in str(diccionario_final['consumo_periodo']).lower():
                            break
                    except Exception:
                        continue

    for indice, linea in enumerate(lista_lineas):
        if diccionario_final['cp_cliente'] == '':
            if diccionario_final['nombre_cliente'].lower() in linea.lower():
                cp = re.search(r"[\d]{5}", lista_lineas[indice+2])
                diccionario_final["cp_cliente"] = cp.group()
                diccionario_final["calle_cliente"] = lista_lineas[indice+1]
                break

                           
    nomi = pgeocode.Nominatim('es')
    diccionario_final["provincia_cliente"] = nomi.query_postal_code(diccionario_final["cp_cliente"])['county_name']              
    diccionario_final["población_cliente"] = nomi.query_postal_code(diccionario_final["cp_cliente"])['place_name']              
    diccionario_final["provincia_comercializadora"] = nomi.query_postal_code(diccionario_final["cp_comercializadora"])['county_name']       
    diccionario_final["población_comercializadora"] = nomi.query_postal_code(diccionario_final["cp_comercializadora"])['place_name']       

    for linea in lista_lineas:
        lista_pueblos = diccionario_final["población_cliente"].split(',')
        for pueblo in lista_pueblos:
            pueblo = pueblo.strip()
            if pueblo.lower() in linea.lower():
                diccionario_final["población_cliente"] = pueblo
                break
    for linea in lista_lineas:
        lista_pueblos = str(diccionario_final["población_comercializadora"]).split(',')
        for pueblo in lista_pueblos:
            pueblo = pueblo.strip()
            if pueblo.lower() in linea.lower():
                diccionario_final["población_comercializadora"] = pueblo
                break

    return diccionario_final

In [11]:

def lectura_y_extraccion(ruta, min, max, pag=0):
    for num_pdf in range (min, max+1):
        lineas_pdf = extaer_pag_pdf(f"pdfs/factura_{num_pdf}.pdf", pag)
        dict_lineas, lista_lineas = pdf_a_dicc_y_lista(lineas_pdf)
        diccionario_final = extraer_info(dict_lineas, lista_lineas)
        return diccionario_final

In [12]:
diccionario_final 

{'nombre_cliente': '',
 'dni_cliente': '',
 'calle_cliente': '',
 'cp_cliente': '',
 'población_cliente': '',
 'provincia_cliente': '',
 'nombre_comercializadora': '',
 'cif_comercializadora': '',
 'dirección_comercializadora': '',
 'cp_comercializadora': '',
 'población_comercializadora': '',
 'provincia_comercializadora': '',
 'número_factura': '',
 'inicio_periodo': '',
 'fin_periodo': '',
 'importe_factura': '',
 'fecha_cargo': '',
 'consumo_periodo': '',
 'potencia_contratada': ''}

In [13]:
lista_lineas

NameError: name 'lista_lineas' is not defined

In [ ]:
for key in diccionario_final.keys():
    if diccionario_final[key] == '':
        lineas_pdf2 = extaer_pag_pdf("pdfs/factura_5.pdf", 1)
        dict_lineas2, lista_lineas2 = pdf_a_dicc_y_lista(lineas_pdf2)
        diccionario_final2 = extraer_info(dict_lineas2, lista_lineas2)
        print(lista_lineas2)
        diccionario_final[key] = diccionario_final2[key]
    else:
        continue    


['Las cuentas claras..................................................................................', 'A continuación le presentamos información detallada sobre su/s contrato/s y factura.', 'Electricidad', 'Nº contrato de acceso', '(ELECTRO DISTRIBUCION DE ALMODOVAR DEL CAMPO SA):', '924599018933', 'Fecha final de contrato: 14.03.2018', 'Datos instalación electricidad', 'Potencia contratada:', '2,668 kW', 'Tarifa de acceso: 2.0A', 'Cuantía Peaje:1 XX,XX €', 'Código CUPS: ES7425796885725250OBEN', 'Información lecturas / consumos', 'Lectura actual:', 'Lectura anterior:', 'Consumo:', 'Llano real', 'Llano real', 'Llano', '14.03.2018', '13.01.2018', '23360 kWh', '23114 kWh', '246 kWh', 'Los costes de energía que se le aplican se determinan en el BOE de', 'fecha 26.12.20XX. Servicio: 97,337%, Permanentes: 0,150%,', 'Diversificación y Seguridad de abastecimiento: 2,513%.', 'Se aplica el precio del alquiler según BOE N 185 de 03.08.20XX.', 'Composición del término de energía: precio fijo (0

In [ ]:
diccionario_final2

{'nombre_cliente': 'SATURNINO MALTÉS NARANJO',
 'dni_cliente': '13714122G',
 'calle_cliente': 'Calle de la Fuente de Antón Merlo ',
 'cp_cliente': '24920',
 'población_cliente': 'Aldea Del Puente, Sahelices Del Payuelo',
 'provincia_cliente': 'León',
 'nombre_comercializadora': 'ENERVER ENERGIA, S.L.',
 'cif_comercializadora': 'B24726044',
 'dirección_comercializadora': 'CALLE 42, POLIGONO EL BONY,',
 'cp_comercializadora': '46740',
 'población_comercializadora': 'Carcaixent',
 'provincia_comercializadora': 'Valencia',
 'número_factura': 'KS8622022848',
 'inicio_periodo': '13.01.2018',
 'fin_periodo': '14.03.2018',
 'importe_factura': '0.0',
 'fecha_cargo': '19.03.2018',
 'consumo_periodo': '',
 'potencia_contratada': '2,668'}

In [ ]:
diccionario_final

{'nombre_cliente': 'SATURNINO MALTÉS NARANJO',
 'dni_cliente': '13714122G',
 'calle_cliente': 'Calle de la Fuente de Antón Merlo ',
 'cp_cliente': '24920',
 'población_cliente': 'Aldea Del Puente, Sahelices Del Payuelo',
 'provincia_cliente': 'León',
 'nombre_comercializadora': 'ENERVER ENERGIA, S.L.',
 'cif_comercializadora': 'B24726044',
 'dirección_comercializadora': 'CALLE 42, POLIGONO EL BONY,',
 'cp_comercializadora': '46740',
 'población_comercializadora': 'Carcaixent',
 'provincia_comercializadora': 'Valencia',
 'número_factura': 'KS8622022848',
 'inicio_periodo': '13.01.2018',
 'fin_periodo': '14.03.2018',
 'importe_factura': '0.0',
 'fecha_cargo': '19.03.2018',
 'consumo_periodo': '',
 'potencia_contratada': '2,668'}

In [ ]:
""" import json
with open("json_meu/factura_5_meu.json", "w") as fp:
    json.dump(diccionario_final, fp, ensure_ascii=False) """

' import json\nwith open("json_meu/factura_5_meu.json", "w") as fp:\n    json.dump(diccionario_final, fp, ensure_ascii=False) '